In [1]:
from qgis.core import *
import pandas as pd
import numpy as np

In [2]:
import sys
import os
from osgeo import ogr
from qgis.core import (
     QgsApplication,
     QgsProcessingFeedback,
     QgsVectorLayer
)
from qgis.analysis import QgsNativeAlgorithms

In [3]:
QgsApplication.setPrefixPath(r'C:\OSGeo4W\apps\qgis', True)
qgs = QgsApplication([], False)
qgs.initQgis()
sys.path.append(r'C:\OSGeo4W\apps\qgis\python\plugins')

In [4]:
import geopandas as gpd
import matplotlib.pyplot as plt
plt.ion()
import xlwings as xw
import time
import seaborn as sns
import math
import statistics
import scipy.stats

import processing
from processing.core.Processing import Processing
Processing.initialize()
QgsApplication.processingRegistry().addProvider(QgsNativeAlgorithms())
#for alg in QgsApplication.processingRegistry().algorithms():
#        print(alg.id(), "--->", alg.displayName())

C:\OSGeo4W\apps\Python39\lib\site-packages\pyproj\__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


False

In [5]:
# living_houses_pts = QgsVectorLayer(r"https://github.com/echo-golf-oscar-romeo/kindergarten_accessibility/blob/c66a8368b58ad126afb051cc72958379104b80fb/living_houses_pts.geojson")
# kindergarten_pts = QgsVectorLayer(r"https://github.com/echo-golf-oscar-romeo/kindergarten_accessibility/blob/5f8e3b92d19082e424ded1010d9ca836e44ba49c/kindergarten_pts.geojson")

living_houses_pts = QgsVectorLayer(r"C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\resid.geojson")
kindergarten_pts = QgsVectorLayer(r"C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\kindergarten_pts.geojson")

In [7]:
from qgis.core import QgsProcessing
from qgis.core import QgsProcessingAlgorithm
from qgis.core import QgsProcessingMultiStepFeedback
from qgis.core import QgsProcessingParameterVectorLayer
from qgis.core import QgsProcessingParameterNumber
from qgis.core import QgsProcessingParameterFileDestination
from qgis.core import QgsProcessingParameterFeatureSink
import processing


class Kindergarten_egor(QgsProcessingAlgorithm):

    def initAlgorithm(self, config=None):
        self.addParameter(QgsProcessingParameterVectorLayer('livinghousescentoids', 'living_houses_centoids', types=[QgsProcessing.TypeVectorPoint], defaultValue=living_houses_pts))
        self.addParameter(QgsProcessingParameterVectorLayer('kindergartencentroids', 'kindergarten_centroids', types=[QgsProcessing.TypeVectorPoint], defaultValue=kindergarten_pts))
        self.addParameter(QgsProcessingParameterNumber('bufferradius', 'buffer_radius', type=QgsProcessingParameterNumber.Integer, minValue=1, defaultValue=300))
        self.addParameter(QgsProcessingParameterFileDestination('Spreadsheet', 'spreadsheet', fileFilter='Microsoft Excel (*.xlsx);;Open Document Spreadsheet (*.ods)', createByDefault=True, defaultValue=None))
        self.addParameter(QgsProcessingParameterFeatureSink('Intersection_raw', 'intersection_raw', optional=True, type=QgsProcessing.TypeVectorAnyGeometry, createByDefault=True, defaultValue=None))

    def processAlgorithm(self, parameters, context, model_feedback):
        # Use a multi-step feedback, so that individual child algorithm progress reports are adjusted for the
        # overall progress through the model
        feedback = QgsProcessingMultiStepFeedback(3, model_feedback)
        results = {}
        outputs = {}

        # Buffer
        alg_params = {
            'DISSOLVE': False,
            'DISTANCE': parameters['bufferradius'],
            'END_CAP_STYLE': 0,  # Round
            'INPUT': parameters['kindergartencentroids'],
            'JOIN_STYLE': 0,  # Round
            'MITER_LIMIT': 2,
            'SEGMENTS': 15,
            'OUTPUT': QgsProcessing.TEMPORARY_OUTPUT
        }
        outputs['Buffer'] = processing.run('native:buffer', alg_params, context=context, feedback=feedback, is_child_algorithm=True)

        feedback.setCurrentStep(1)
        if feedback.isCanceled():
            return {}

        # Join attributes by location
        alg_params = {
            'DISCARD_NONMATCHING': False,
            'INPUT': outputs['Buffer']['OUTPUT'],
            'JOIN': parameters['livinghousescentoids'],
            'JOIN_FIELDS': [''],
            'METHOD': 0,  # Create separate feature for each matching feature (one-to-many)
            'PREDICATE': [1],  # contains
            'PREFIX': 'j_',
            'OUTPUT': parameters['Intersection_raw']
        }
        outputs['JoinAttributesByLocation'] = processing.run('native:joinattributesbylocation', alg_params, context=context, feedback=feedback, is_child_algorithm=True)
        results['Intersection_raw'] = outputs['JoinAttributesByLocation']['OUTPUT']

        feedback.setCurrentStep(2)
        if feedback.isCanceled():
            return {}

        # Export to spreadsheet
        alg_params = {
            'FORMATTED_VALUES': False,
            'LAYERS': outputs['JoinAttributesByLocation']['OUTPUT'],
            'OVERWRITE': True,
            'USE_ALIAS': False,
            'OUTPUT': parameters['Spreadsheet']
        }
        outputs['ExportToSpreadsheet'] = processing.run('native:exporttospreadsheet', alg_params, context=context, feedback=feedback, is_child_algorithm=True)
        results['Spreadsheet'] = outputs['ExportToSpreadsheet']['OUTPUT']
        print(results['Spreadsheet'])
        return results

    def name(self):
        return 'kindergarten_egor'

    def displayName(self):
        return 'kindergarten_egor'

    def group(self):
        return 'egor'

    def groupId(self):
        return 'egor'

    def createInstance(self):
        return Kindergarten_egor()

In [24]:
base = Kindergarten_egor()
buffer_radius = 300
intersection_raw = r"C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\trash\trash_0001.geojson" 
table = r"C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\trash\trash_0001.xlsx"
parameters = {
    'bufferradius': buffer_radius,
    'kindergartencentroids': kindergarten_pts,
    'livinghousescentoids': living_houses_pts,
    'Intersection_raw': intersection_raw,
    'Spreadsheet': table
}
context = QgsProcessingContext()
feedback = QgsProcessingFeedback()
base.processAlgorithm(parameters, context=context, model_feedback=feedback)

brand_new_data = pd.read_excel(r"C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\trash\trash_0001.xlsx")
xb = xw.Book(r"C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\Calculator_kindergarten_to_place.xlsx")
sheet = xb.sheets("intersection_raw")
sheet.clear()
sheet.range("A1", "K1000").value = brand_new_data
xb.save()
    
# do it again for one more house
    
base_1 = Kindergarten_egor()
#df1 = pd.DataFrame(columns=['id', 'endowment'])
for i in range(3488, 7374, 1):   
    intersection_raw = r"C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_000%s.geojson" % i
    table = r"C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_000%s.xlsx" % i
    parameters = {
        'bufferradius': buffer_radius,
        'kindergartencentroids': QgsVectorLayer(r"C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\id_%s.geojson" % i),
        'livinghousescentoids': living_houses_pts,
        'Intersection_raw': intersection_raw,
        'Spreadsheet': table
    }
    context = QgsProcessingContext()
    feedback = QgsProcessingFeedback()
    base_1.processAlgorithm(parameters, context=context, model_feedback=feedback)

    brand_new_data = pd.read_excel(r"C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_000%s.xlsx" % i)
    xb = xw.Book(r"C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\Calculator_kindergarten_to_place.xlsx")
    sheet = xb.sheets("intersection_raw(1)")
    sheet.clear()
    sheet.range("A1", "K1000").value = brand_new_data
    xb.save()
    
    endowment = round(pd.read_excel(r"C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\Calculator_kindergarten_to_place.xlsx", sheet_name = '10', engine="openpyxl")["Unnamed: 1"].loc[0], 2)
    temp = pd.DataFrame([[i, endowment]],columns=['id', 'endowment'])
    df1 =  pd.concat([df1, temp])
    print(endowment)
#print(df1['endowment'])

C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\trash\trash_0001.xlsx
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003488.xlsx
47.53
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003489.xlsx
47.56
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003490.xlsx
47.57
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003491.xlsx
47.59
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003492.xlsx
47.59
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003493.xlsx
47.61
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003494.xlsx
47.62
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003495.xlsx
47.64
C:\Users\egorb\Documents\ГП\2 курс\П

47.7
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003562.xlsx
47.71
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003563.xlsx
47.67
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003564.xlsx
47.62
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003565.xlsx
47.63
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003566.xlsx
47.62
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003567.xlsx
47.64
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003568.xlsx
47.66
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003569.xlsx
47.67
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003570.xlsx
47.61
C:\Us

47.3
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003637.xlsx
47.31
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003638.xlsx
47.37
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003639.xlsx
47.38
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003640.xlsx
47.44
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003641.xlsx
47.45
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003642.xlsx
47.48
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003643.xlsx
47.49
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003644.xlsx
47.51
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003645.xlsx
47.52
C:\Us

47.57
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003712.xlsx
47.5
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003713.xlsx
47.47
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003714.xlsx
47.52
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003715.xlsx
47.31
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003716.xlsx
47.34
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003717.xlsx
47.34
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003718.xlsx
47.37
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003719.xlsx
47.34
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003720.xlsx
47.37
C:\Us

47.88
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003787.xlsx
47.88
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003788.xlsx
47.91
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003789.xlsx
47.91
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003790.xlsx
47.93
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003791.xlsx
47.96
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003792.xlsx
47.98
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003793.xlsx
47.94
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003794.xlsx
47.97
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0003795.xlsx
47.9
C:\Us

...

47.73
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007012.xlsx
47.67
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007013.xlsx
47.67
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007014.xlsx
47.63
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007015.xlsx
47.44
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007016.xlsx
47.48
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007017.xlsx
47.51
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007018.xlsx
47.48
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007019.xlsx
47.53
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007020.xlsx
47.53
C:\U

47.55
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007087.xlsx
47.59
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007088.xlsx
47.66
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007089.xlsx
47.7
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007090.xlsx
47.7
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007091.xlsx
47.68
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007092.xlsx
47.7
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007093.xlsx
47.66
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007094.xlsx
47.66
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007095.xlsx
47.65
C:\User

47.59
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007162.xlsx
47.65
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007163.xlsx
47.62
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007164.xlsx
47.56
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007165.xlsx
47.56
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007166.xlsx
47.54
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007167.xlsx
47.59
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007168.xlsx
47.6
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007169.xlsx
47.75
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007170.xlsx
47.72
C:\Us

48.12
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007237.xlsx
48.11
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007238.xlsx
47.57
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007239.xlsx
47.54
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007240.xlsx
47.5
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007241.xlsx
47.68
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007242.xlsx
47.76
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007243.xlsx
47.76
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007244.xlsx
47.75
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007245.xlsx
47.72
C:\Us

47.67
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007312.xlsx
47.68
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007313.xlsx
47.7
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007314.xlsx
47.84
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007315.xlsx
47.53
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007316.xlsx
47.51
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007317.xlsx
47.69
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007318.xlsx
47.69
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007319.xlsx
47.68
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\new_kindergarten\trash\trash_0007320.xlsx
47.7
C:\Use

In [13]:
print(df1['endowment'])

0    47.86
0    47.86
0    47.86
0    47.88
0    47.88
     ...  
0    47.47
0    47.47
0    47.48
0     47.5
0    47.51
Name: endowment, Length: 3487, dtype: object


In [26]:
df1.to_excel(r"C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\Calculator_new_kindergarten.xlsx", sheet_name='new_1')